### 第十六章 面向对象程序设计

#### 本章内容

1. 封装
2. 继承
3. 多态
4. 重载

#### 1. 封装

封装（Encapsulation）是面向对象程序设计的核心思想之一。封装指的是将数据（属性）以及处理数据的代码（方法）组织在一起，形成一个“独立的模块”，即“类”。在日常生活中，封装的例子随处可见。例如，手机可以打电话、发信息和上网，其内部结构如何实现，用户并不需要了解，只需通过手机的功能界面，就能顺利完成操作。手机把复杂的功能细节“封装”在一个简单的外壳和接口之下，极大地提升了易用性和安全性。

在编程中，封装的应用带来如下主要优势：

- **隐藏实现细节**：外部只需通过有限的方法与对象交互，无需关心其内部实现，大大提升了代码的安全性与可维护性。
- **组织代码结构**：通过将相关数据与操作整合在一起，有助于模块化编程，让整体设计更清晰易懂。
- **限制数据访问**：通过在类中设定数据的访问权限，可以防止数据被意外修改，保护数据的正确性。

本章将通过设计和完善“彩色气泡生态系统”程序帮助读者理解面向对象四大核心思想。最终，这个应用将实现一个视觉艺术化的气泡生态系统：在屏幕上能够看到多个形态各异、动态运动、富有生命力的彩色气泡，包括基础气泡和进化气泡的互动行为。随着本章内容的推进，程序会逐步引入更复杂的继承、多态及重载等特性。

通过合理设计气泡类，将每个气泡的属性（如位置、颜色、大小等）与功能（如显示和移动）有效地封装在一个独立的类结构中，从而为后续扩展各种气泡和行为打下基础。

基于封装原则，可以先对Bubble类进行结构上的设计。  
如下为Bubble类的属性和方法声明（仅为设计结构，并未实现具体方法逻辑）：

```python
class Bubble:
    def __init__(self, x, y, r, col):
        # 初始化气泡的位置、半径、颜色等属性
        pass
    
    def show(self):
        # 负责将气泡绘制到屏幕上的方法
        pass
```

通过这种方式，每个气泡对象都可以存储和管理属于自己的数据，并只通过`move()`与`show()`这样的“外部接口”与外部交互，而无需暴露其内部细节。这正是封装思想在具体编程实践中的体现。

结合前面类的设计，读者可以先实现一个最基础的气泡可视化程序。在这个版本中，只完成属性的封装与显示方法，不涉及复杂的行为逻辑。

In [33]:
# 在Jupyter代码单元中强制重启内核
import IPython
IPython.get_ipython().run_line_magic('reset', '-f')

import py5

class Bubble:
    def __init__(self, x, y, r, col):
        self.x = x
        self.y = y
        self.r = r
        self.col = col

    def move(self):
        # 本节暂不实现移动，仅为接口声明
        pass

    def show(self):
        py5.no_stroke()
        py5.fill(*self.col)
        py5.ellipse(self.x, self.y, self.r * 2, self.r * 2)

bubbles = []

def setup():
    py5.size(600, 400)
    py5.background(255)
    py5.text_font(py5.create_font("Noto Sans Thin", 16))
    for _ in range(10):
        x = py5.random(50, py5.width - 50)
        y = py5.random(50, py5.height - 50)
        r = py5.random(15, 30)
        col = (py5.random(100, 255), py5.random(100, 255), py5.random(100, 255), 180)
        bubbles.append(Bubble(x, y, r, col))

def draw():
    py5.background(255)
    for bubble in bubbles:
        bubble.show()

py5.run_sketch()


py5 encountered an error in your code:

File "C:\Users\PXQ\AppData\Local\Temp\ipykernel_14088\902672819.py", line 39, in draw
    36   def draw():
    37       py5.background(255)
    38       for bubble in bubbles:
--> 39           bubble.show()
    ..................................................
     bubble.show = <method 'Bubble.show' of <__main__.Bubble object at 0x000001
                    EC53CBA2C0> 1912567038.py:18>
    ..................................................

File "C:\Users\PXQ\AppData\Local\Temp\ipykernel_14088\1912567038.py", line 20, in show
    18   def show(self):
    19       py5.no_stroke()
--> 20       py5.fill(*self.col)
    21       py5.ellipse(self.x, self.y, self.r * 2, self.r * 2)
    ..................................................
     self.col = (171.0179761233597, 106.34880747617792, 246.83491740988106, 
                 180, )
    ..................................................

NameError: name 'py5' is not defined


#### 2. 继承

在完成了基本的封装之后，程序中已经有了功能完善的气泡类（Bubble）。但随着需求的增长，仅有这种“基础气泡”远远不够。例如，我们想让气泡中“部分成员”拥有更丰富的行为和属性，例如会闪烁、能变色，甚至具备个性化的交互。这时，每种新气泡都从头开始定义的话，难免造成大量重复，程序不仅臃肿，而且难以维护。

在面对这些需求时，面向对象体系下的“继承”思想发挥了巨大的力量。它可以让已经存在的类成为其他新类的“蓝本”，新类自动获得父类全部已有功能，并能在其基础上自主扩展。

继承并不是编程世界的专属，现实生活中随处可见。例如，所有哺乳动物都有心脏和肺（这些是共同的“父辈特征”），但鲸鱼、袋鼠和人又各自拥有独特技能。又如，家用轿车和电动车，是在“汽车”这一本体之上，分别加入了自己的新属性和新设计。各自的特别之处，正是在继承基础上的创新和增长。

在面向对象编程中，继承是一种机制，允许新建的类自动获得已有类的全部属性与方法，新建的类称为“子类”，已有的类称为“父类”或“基类”。

- **父类（基类）**：被继承的类，定义通用属性和方法。
- **子类（派生类）**：继承父类后，可以获得父类已有能力，并扩展自己独有的能力。

这种继承关系，避免了代码的重复，让代码层次更加清晰。例如，所有“气泡”都有“移动”和“显示”的能力，如果要增加一种“特殊气泡”，其实只需要在基础气泡的基础上增加自己独有的内容就可以了。

Python通过在类定义括号中写上父类名，便能实现继承。子类自动获得所有父类的属性和方法。

In [34]:
class Parent:
    def say_hello(self):
        print("Hello from Parent")

class Child(Parent):    # Child继承自Parent
    def say_hi(self):
        print("Hi from Child")

# 使用
c = Child()
c.say_hello()  # 输出：Hello from Parent
c.say_hi()     # 输出：Hi from Child

Hello from Parent
Hi from Child


通过如上定义，`Child`类实例既可以调用自己的方法（say_hi），也可以直接使用父类的方法（say_hello），无需重复实现。

在“彩色气泡生态系统”项目中，已经有了基本的Bubble类，定义了所有气泡公有的属性（如位置、半径、颜色）和方法（如move、show）。  
随着需求拓展，需要在此基础上，派生出“特殊气泡”类（SpecialBubble），用于扩展新特性，例如用于支持闪烁、独特交互等。

- **父类（基类）**：Bubble，负责普通气泡的基本属性和行为。
- **子类**：SpecialBubble，继承自Bubble，自动获得Bubble的一切特征，并在其基础上增加新的属性与方法（比如`alpha`用于实现闪烁，`twinkle()`为独有方法），但并未修改父类已存在的方法逻辑，不涉及重写。

这种结构便于组织和扩展气泡家族，充分发挥继承的优势。

In [35]:
# 在Jupyter代码单元中强制重启内核
import IPython
IPython.get_ipython().run_line_magic('reset', '-f')

import py5

class Bubble:
    def __init__(self, x, y, r, col):
        self.x = x
        self.y = y
        self.r = r
        self.col = col

    def move(self):
        self.x += py5.random(-0.5, 0.5)
        self.y += py5.random(-0.5, 0.5)

    def show(self):
        py5.no_stroke()
        py5.fill(*self.col)
        py5.ellipse(self.x, self.y, self.r * 2, self.r * 2)

class SpecialBubble(Bubble):
    def twinkle(self): 
        self.col = (self.col[0], self.col[1], self.col[2], py5.random(20, 255))

bubbles = []
special_bubbles = []

def setup():
    py5.size(600, 400)
    py5.background(255)
    py5.text_font(py5.create_font("Noto Sans Thin", 16))
    for _ in range(2):
        x = py5.random(50, py5.width - 50)
        y = py5.random(50, py5.height - 50)
        r = py5.random(15, 30)
        col = (py5.random(80,250), py5.random(80,250), py5.random(80,250), 180)
        bubbles.append(Bubble(x, y, r, col))
    for _ in range(2):
        x = py5.random(50, py5.width - 50)
        y = py5.random(50, py5.height - 50)
        r = py5.random(18, 34)
        col = (py5.random(100,255), py5.random(100,255), py5.random(100,255), 180)
        special_bubbles.append(SpecialBubble(x, y, r, col))

def draw():
    py5.background(255)
    for bubble in bubbles:
        bubble.move()
        bubble.show()
    for special in special_bubbles:
        special.move()
        special.show()
        special.twinkle()  # 单独显示特效行为（twinkle）

    py5.fill(40, 80)
    py5.text("普通气泡", 20, 30)
    py5.text("特殊气泡（闪烁）", 420, 30)

py5.run_sketch()


Sketch is already running. To run a new sketch, exit the running sketch first.


#### 3. 多态

在继承实现后，彩色气泡生态系统已经拥有了基础气泡和特殊气泡两种类型。每个气泡不仅拥有自己的属性，也共享了基础的行为，同时也可能有着各自的扩展功能。这时会面临一个新问题：**如何在不区分类别的前提下，统一操作所有类型的气泡？**例如：在屏幕每一帧更新时，希望让所有气泡都移动、都显示，但又不想为每类气泡写重复的循环结构，也不希望手动判断每一个对象到底是什么类型。

这时，多态（polymorphism）的思想应运而生。**多态允许我们用统一的接口“操作多种实际类型”的对象，而无需关心它们内部的具体实现细节**。如此，系统在扩展和维护时就会变得极为简便和灵活。

多态其实不仅仅属于编程术语，在现实世界中同样随处可见。比如：

- 指挥家用同一个动作“挥棒”，小提琴、钢琴、长笛都在各自“用乐器”响应这个动作，但实际发出的声音和技巧是不同的。这个“挥棒”就是统一接口，具体表现由每个乐手决定。
- 智能手机上的“拍照按钮”。用户只需按下同一个按钮，无论是后置、前置还是广角镜头，手机会自动调用对应的拍照方式，这就是“一种操作，多种响应”。

在编程中，多态允许我们编写“统一命令”，让程序自动分发到各自的对象，由它们自己决定怎样实现细节。

多态通常依托于继承结构。父类规定了所有子类都应当具备的操作接口（比如move和show方法），子类可以选择保留父类的实现，也可以进行方法重写（override）。无论每个对象的真实类型如何，只要在需要的时候调用接口方法，Python（乃至大多数面向对象语言）都会自动根据对象实际类型，选择合适的方法实现。

- **统一接口，动态响应**：无需知道对象真实类型，调用相同方法，实际响应由该对象的类决定。
- **便于扩展和维护**：增加新类型对象时，无需修改主流程代码，只需保证实现了必要接口即可。
- **减少重复逻辑，提高代码可读性和灵活性**。

最直接的多态实现，就是将各种“子类对象”混合存储，然后统一调用方法：

In [36]:
class Animal:
    def speak(self):
        print("Generic animal sound")

class Dog(Animal):
    def speak(self):
        print("Woof!")

class Cat(Animal):
    def speak(self):
        print("Meow!")

animals = [Dog(), Cat(), Animal()]
for a in animals:
    a.speak()  # 输出各自不同的叫声，自动分派

Woof!
Meow!
Generic animal sound



即使`animals`这个列表包含多种动物对象，同一个`speak()`调用接口，却能得到不同实际的“叫声”。这就是多态。

回到彩色气泡生态系统，前面的设计中已经拥有`Bubble`（基础气泡）和`SpecialBubble`（特殊气泡）两种类型。每种气泡都各自拥有move和show方法，但特殊气泡通过方法重写（override）的方式，拥有了不同行为（例如闪烁移动、特殊显示）。

在主程序中，可以不分别存储基础气泡和特殊气泡，而是把它们全部装进同一个“气泡列表”里。那么，**只需一行循环遍历，通过统一的move和show接口，程序会自动根据气泡实际类型调用相应的方法，实现不同的视觉表现**，这是多态最经典的用法。

- `Bubble`类：定义move和show方法（普通的移动和显示）
- `SpecialBubble`类：继承自Bubble，重写move和show方法，实现闪烁与特殊视觉效果
- 程序：用一个列表统一存放所有种类的气泡，只需统一调用方法即可完成全部细节操作

这样设计后，后续即便需要扩展更多气泡类型（例如发光气泡、运动轨迹特殊气泡等），都不必修改主流程，充分利用了“统一接口”的好处。

In [37]:
# 在Jupyter代码单元中强制重启内核
import IPython
IPython.get_ipython().run_line_magic('reset', '-f')

import py5

class Bubble:
    def __init__(self, x, y, r, col):
        self.x = x
        self.y = y
        self.r = r
        self.col = col

    def move(self):
        self.x += py5.random(-2, 2)
        self.y += py5.random(-2, 2)

    def show(self):
        py5.no_stroke()
        py5.fill(*self.col)
        py5.ellipse(self.x, self.y, self.r * 2, self.r * 2)

class SpecialBubble(Bubble):
    def __init__(self, x, y, r, col):
        super().__init__(x, y, r, col)
        self.alpha = py5.random(80, 255)
        self.twinkle_speed = py5.random(2, 5)
        self.alpha_direction = 1

    def move(self):
        # 特殊气泡的移动方式与普通气泡可能不同
        self.x += py5.random(-3, 3)
        self.y += py5.random(-3, 3)
        # 同时动态调整透明度
        self.alpha += self.twinkle_speed * self.alpha_direction
        if self.alpha > 255:
            self.alpha = 255
            self.alpha_direction = -1
        if self.alpha < 80:
            self.alpha = 80
            self.alpha_direction = 1

    def show(self):
        py5.no_stroke()
        col_with_alpha = (self.col[0], self.col[1], self.col[2], self.alpha)
        py5.fill(*col_with_alpha)
        py5.ellipse(self.x, self.y, self.r * 2, self.r * 2)

bubbles = []

def setup():
    py5.size(600, 400)
    py5.background(255)
    py5.text_font(py5.create_font("Noto Sans Thin", 16))
    for _ in range(6):
        x = py5.random(50, py5.width - 50)
        y = py5.random(50, py5.height - 50)
        r = py5.random(18, 30)
        col = (py5.random(80,250), py5.random(80,250), py5.random(80,250), 180)
        bubbles.append(Bubble(x, y, r, col))
    for _ in range(4):
        x = py5.random(50, py5.width - 50)
        y = py5.random(50, py5.height - 50)
        r = py5.random(18, 32)
        col = (py5.random(120,255), py5.random(100,255), py5.random(100,255), 180)
        bubbles.append(SpecialBubble(x, y, r, col))

def draw():
    py5.background(255)
    for bubble in bubbles:
        bubble.move()   # 多态统一调用，不必区分类型
        bubble.show()
    py5.fill(40, 80)
    py5.text("多态演示：多种气泡类型统一管理", 20, 30)

py5.run_sketch()

py5 encountered an error in your code:

File "C:\Users\PXQ\AppData\Local\Temp\ipykernel_14088\2072560526.py", line 70, in draw
    68   def draw():
    69       py5.background(255)
--> 70       for bubble in bubbles:
    71           bubble.move()   # 多态统一调用，不必区分类型

NameError: name 'bubbles' is not defined


#### 4. 重载

随着气泡生态系统逐步复杂化，除了需要封装、继承和多态之外，有时还希望为程序中的对象提供更加“自然、直观”的操作方式。重载（Overloading）便是面向对象体系中用于增强表达力的重要机制。

**重载**指的是：为同一个名称定义多种不同的实现方式，使同一个操作符或函数名能针对不同输入或上下文表现出不同行为。
在静态语言（如C++/Java）中，常有“函数重载”（同名不同参），但在Python里，重载的重点通常体现在“特殊方法”（魔术方法）的实现，也即“运算符重载”和“内置函数重载”。

在Python中，多个对象可以通过重载某些特殊方法，让行为更符合人类习惯。例如，能够直接用+运算符让两个对象“相加”，或者用len()获取对象长度。

重载的出现，使得编写和阅读代码都更加简洁、直观。

Python为自定义类预留了许多**特殊方法**（Magic Methods），允许开发者通过重写这些特殊方法，来让对象支持各种原生语法操作（如运算、比较、转字符串、内置函数等）。  

常用可重载特殊方法一览：

| 方法名        | 作用             | 举例触发                      |
|:-------------:|------------------|-------------------------------|
| `__init__`    | 构造器           | `Bubble()`                    |
| `__str__`     | 字符串表示        | `print(obj)`                  |
| `__repr__`    | 官方字符串表示    | `repr(obj)`                   |
| `__len__`     | 长度             | `len(obj)`                    |
| `__getitem__` | 下标访问         | `obj[0]`                      |
| `__setitem__` | 下标赋值         | `obj[0] = ...`                |
| `__add__`     | 加法运算         | `obj1 + obj2`                 |
| `__eq__`      | 等值比较         | `obj1 == obj2`                |
| `__call__`    | 可调用对象        | `obj()`                       |

In [38]:

class NumberBox:
    def __init__(self, value): self.value = value
    def __str__(self): return f"Box:{self.value}"
    def __add__(self, other): return NumberBox(self.value + other.value)

a = NumberBox(3)
b = NumberBox(5)
print(a)             # Box:3 (__str__触发)
print(a + b)         # Box:8 (__add__和__str__都触发)

Box:3
Box:8


Python不像C++/Java支持同名不同参数的**真正**重载，但可以通过**默认参数**、`*args, **kwargs`、类型判断等模拟。

In [39]:
class Greeter:
    def greet(self, name=None):
        if name is None:
            print("Hello, stranger.")
        else:
            print("Hello,", name)
g = Greeter()
g.greet()           # Hello, stranger.
g.greet("Alice")    # Hello, Alice


Hello, stranger.
Hello, Alice


在“彩色气泡生态系统”项目中，首先，通过**运算符重载**，可以让两个气泡用加号直接“融合”为一个新气泡——这种表达极大提升了操作的艺术性和自然感。其次，通过**参数重载模拟**，Bubble类的方法（如move）可根据传入参数的不同，表现为不同的运动方式，既可以随机移动，也可以按指定方向移动，让操作更具多样性和灵活性。


In [40]:
# 在Jupyter代码单元中强制重启内核
import IPython
IPython.get_ipython().run_line_magic('reset', '-f')

import py5
import math

class Bubble:
    def __init__(self, x, y, r, col):
        self.x = x
        self.y = y
        self.r = r
        self.col = col  # (R, G, B, A)

    # 运算符重载——融合：bubble1 + bubble2
    def __add__(self, other):
        # 融合中心
        new_x = (self.x + other.x) / 2
        new_y = (self.y + other.y) / 2
        # 融合半径（面积和再开方）
        new_area = math.pi * self.r**2 + math.pi * other.r**2
        new_r = math.sqrt(new_area / math.pi)
        # 融合颜色（RGBA均值）
        new_col = tuple(int((a + b) / 2) for a, b in zip(self.col, other.col))
        return Bubble(new_x, new_y, new_r, new_col)

    # 支持两种移动方式：无参=随机漂移，有参=定向
    def move(self, dx=None, dy=None):
        if dx is not None and dy is not None:
            self.x += dx
            self.y += dy
        else:
            self.x += py5.random(-2, 2)
            self.y += py5.random(-2, 2)

    def show(self):
        py5.no_stroke()
        py5.fill(*self.col)
        py5.ellipse(self.x, self.y, self.r * 2, self.r * 2)

class SpecialBubble(Bubble):    # 继承
    def __init__(self, x, y, r, col):
        super().__init__(x, y, r, col)
        self.alpha = py5.random(60, 220)         # 闪烁的透明度
        self.twinkle_speed = py5.random(2, 5)
        self.alpha_dir = 1

    # 重写move，实现闪烁与更快漂移
    def move(self, dx=None, dy=None):
        super().move(dx, dy)
        # 闪烁效果
        self.alpha += self.twinkle_speed * self.alpha_dir
        if self.alpha > 240:
            self.alpha = 240
            self.alpha_dir = -1
        if self.alpha < 60:
            self.alpha = 60
            self.alpha_dir = 1

    # 重写show方法，支持动态alpha显示
    def show(self):
        py5.no_stroke()
        c = (self.col[0], self.col[1], self.col[2], int(self.alpha))
        py5.fill(*c)
        py5.ellipse(self.x, self.y, self.r * 2, self.r * 2)
        py5.stroke(80, 100)
        py5.no_fill()
        py5.ellipse(self.x, self.y, self.r * 2.7, self.r * 2.7)

bubbles = []

def setup():
    py5.size(700, 420)
    py5.background(255)
    py5.text_font(py5.create_font("Noto Sans Thin", 16))
    # 生成5个普通气泡，3个特殊气泡，全部放入同一个列表
    for _ in range(5):
        x = py5.random(70, py5.width - 70)
        y = py5.random(70, py5.height - 70)
        r = py5.random(15, 30)
        col = (py5.random(80,220), py5.random(120,255), py5.random(150,255), 180)
        bubbles.append(Bubble(x, y, r, col))
    for _ in range(3):
        x = py5.random(90, py5.width - 90)
        y = py5.random(90, py5.height - 90)
        r = py5.random(18, 32)
        col = (py5.random(150,255), py5.random(100,200), py5.random(100,200), 140)
        bubbles.append(SpecialBubble(x, y, r, col))

    # 演示融合与定向移动：
    # bubbles[0]和bubbles[1]融合，把新气泡加入列表
    fused = bubbles[0] + bubbles[1]
    bubbles.append(fused)
    # 让第一个气泡定向移动
    bubbles[0].col = (255, 0, 0, 255) # 变成大红色，以便演示

def draw():
    py5.background(255)
     # 让第一个气泡定向移动
    bubbles[0].move(0.1, -0.3)   # 参数重载模拟
    bubbles[0].show()
    for i in range(len(bubbles) - 1):
        bubbles[i + 1].move()
        bubbles[i + 1].show()
    py5.fill(50, 90)
    py5.text("继承：SpecialBubble是Bubble子类", 16, 30)
    py5.text("多态：统一列表动态操作", 16, 60)
    py5.text("重载：+为融合，move支持两种方式", 16, 90)

py5.run_sketch()

#### 本章总结

##### 本章知识点汇总

1. **封装**：将属性和方法组织在一起形成类，隐藏实现细节，仅通过接口与外部交互，从而提升安全性和模块化。
2. **继承**：新类（子类）自动拥有父类的属性与方法，并可以扩展独有功能，实现代码复用与扩展。
3. **多态**：不同类型对象通过统一接口调用方法，根据实际类型自动执行各自的实现，增强程序灵活性和可扩展性。
4. **重载**：允许同名操作符或方法针对不同参数或上下文表现为不同功能，使代码表达更自然、简洁。

##### 课后练习

1. 自动售货机不仅能够卖饮料，还能卖零食和车票。每种商品虽然类别不同，但都能“购买”、显示价格。请结合OOP思想，指出哪些设计体现了“继承”以及“多态”？
2. 智能门锁系统对外只暴露“录入指纹”“验证指纹”“开/关锁”几个简单按钮。你无法看到锁芯的具体工作原理和指纹比对算法——这种设计体现了哪个OOP概念？请说明其优点。
3. 在同一个银行APP中，“+”操作在不同功能区，既能实现“新建账户”，也能实现“转账金额相加”，有时还能用于“添加银行卡”。请指出这属于哪个OOP概念，并谈谈对用户界面的好处。
4. 一个线上课堂系统里，老师和学生都要“登录”，但老师登录后能管理教学资料、布置作业，而学生登录后只能查看作业并提交作品。同一个“登录”入口，却根据账户类型响应不同——请说明此处多态的体现。
5. 你拍摄的照片文件，在不同软件中点击同样的“编辑”按钮，有的弹出图像处理菜单，有的则进入裁剪界面，但操作均为“编辑”。从OOP角度分析这是什么现象，并举例说明多态的实际好处。
6. 请用Python定义一个User类，实现用户名和密码的私有（封装），并提供公有方法用于密码验证。
7. 请写出一个Animal类及其子类Bird和Dog，要求Animal有speak方法，子类speak方法输出各自叫声（如Dog输出“汪汪”，Bird输出“啾啾”）。
8. 写一个Python函数greet(person)，它能接收Student和Teacher对象，调用它们各自实现的introduce方法，实现多态效果；并演示各传入一次时的输出。
9. 完善Vector类，使得a+b可以返回向量相加结果，并实现重载str方法，使Vector对象直接可打印成“(x, y)”格式。
10. 用py5实现一个“多功能按钮”对象Button，能够根据传入参数显示不同形状（如圆、方），点击时改变颜色。要求实现封装和重载思想。
11. 设计一个交通工具可视化系统，定义Vehicle为基类，Car和Boat为子类，每个对象可在画面上移动并用不同造型显示，用多态一起管理和绘制它们。
12. 实现一个智能家居场景模拟，基类Device，子类有Light、Curtain等，不同设备支持不同动画效果。要求：所有设备都可存入同一列表遍历显示，多种类型设备可用“+”融合为场景，用函数模拟接口，充分体现面向对象四要素。

##### 扩展知识

随着信息技术和数字艺术的发展，**面向对象编程（OOP）**不仅是计算机科学入门的核心方法，也成为组织复杂系统、实现艺术与工程跨界创作的重要支撑。OOP通过“封装”保护细节、“继承”实现模块复用、“多态”让同一接口表现多样性、“重载”丰富操作灵活性的方式，让编程不仅高效，而且具有极强的可扩展性和易维护性。

但OOP并不是唯一的现代编程思想。近年来，敏捷开发成为软件工程的重要补充，它鼓励团队成员频繁交流、快速迭代和持续反馈，帮助作品在需求变化时自如调整。而**函数式编程**强调数据不可变、纯函数和链式处理，常被用于数据密集型、生成艺术、机器学习等对运算结果唯一性和可预测性有较高要求的场景。两者与OOP结合，可以提升系统健壮性和创新能力。

此外，**组合优于继承**、**响应式编程**、**模块化/组件化开发**、**微服务架构**、**领域驱动设计（DDD）**等新兴思想也逐步成为主流。组合思路让不同功能像“积木”一样动态拼装和切换，适合快速原型设计和交互体验重构；响应式技术让系统能自动处理变化的数据流，广泛用于全息投影、实时数据可视化等媒体艺术场景；领域驱动设计则强调从实际问题出发建立直观的抽象模型，为复杂系统和大型艺术装置的设计提供了坚实支撑。

在更深的理论层面，数学与算法为OOP拓展更广阔的创造空间。**数学建模**把现实世界的问题抽象为数学和算法对象——在交互艺术装置、动态音乐、数据可视化和大规模行为仿真时，合理建模常常决定了作品的表现力和“灵魂”；**离散数学**（如图论、集合论、逻辑、组合学）是理解程序组成和数据结构的理论基础，支持算法优化和复杂互动关系刻画；**博弈论**关注多方策略和利益平衡，适合群体行为艺术和带有策略性的互动体验设计。更前沿的主题，如**计算复杂性**、**人工智能基础**、**概率与统计**、**并行/分布式计算**、**量子计算与编码理论**等，都或直接或间接推动了艺术与编程工具的进步，让跨学科合作成为可能。

**总结来看，OOP融汇了结构化、系统化与艺术表达的优势，也是理解更多先进开发理念的跳板。无论面对何种复杂项目，将OOP与敏捷开发、函数式编程、响应式思维等灵活结合，再借助数理知识和现代工程理论，都能帮助你突破认知边界、实现更富表现力和生命力的创作。**

- [面向对象程序设计（Python） | 浙江大学](https://www.icourse163.org/course/ZJU-93001)  
- [敏捷开发与Scrum | 西安电子科技大学](https://www.icourse163.org/course/XDU-1206619810)  
- [函数式编程基础 | 浙江大学](https://www.icourse163.org/course/ZJU-93005)  
- [Wolfram Demonstrations Project（部分支持中文检索）](https://demonstrations.wolfram.com/)
大量计算机科学、数学建模、博弈论等可视化演示，适合理解交互和算法原理。

##### 练习题提示

1. **继承**：饮料、零食、车票都是商品（商品为父类，各自为子类）；**多态**：多种商品对象都实现了“购买”“显示价格”等统一接口，实际操作因种类不同效果不同。
2. **封装**。优点：简化操作，保证数据和逻辑安全，便于维护和升级。
3. **重载**。不同场景同一按钮简化了界面，提升了用户体验和操作效率。
4. **多态**。相同的“登录”入口，老师、学生分别获得不同功能，系统扩展一致性更强。
5. **多态**。好处：同一操作指令可以支持不同类型数据/对象，用户体验一致，系统更灵活。
6.  
    ```python
    class User:
        def __init__(self, username, password):
            self.__username = username
            self.__password = password
        def verify(self, password):
            return self.__password == password
    ```
7.  
    ```python
    class Animal:
        def speak(self):
            print("动物叫声")
    class Bird(Animal):
        def speak(self):
            print("啾啾")
    class Dog(Animal):
        def speak(self):
            print("汪汪")
    ```
8.  
    ```python
    class Student:
        def introduce(self):
            print("我是学生")
    class Teacher:
        def introduce(self):
            print("我是老师")
    def greet(person):
        person.introduce()
    greet(Student())
    greet(Teacher())
    ```
9.  
    ```python
    class Vector:
        def __init__(self, x, y):
            self.x = x
            self.y = y
        def __add__(self, other):
            return Vector(self.x + other.x, self.y + other.y)
        def __str__(self):
            return f"({self.x}, {self.y})"
    ```
10.  
    - 定义Button类，把形状、颜色作为私有成员，提供改变颜色方法。重载显示（draw）方法，根据参数输出不同形状。
    - 提示：构造时参数shape为"circle"或"square"，draw方法判断形状；clicked时调用change_color方法。
11.  
    - Vehicle有move和draw方法，Car和Boat实现draw表现不同造型。所有对象存入同一列表，循环调用draw、move。即体现多态。
12.  
    - Device为父类，派生Light、Curtain等子类，实现各自动画。所有对象存入列表遍历显示。“+”运算重载为场景Device。封装属性，函数形式暴露操作接口。可通过按钮模拟设备开/关、组合等。
